In [1]:
import pandas as pd
from sqlalchemy import create_engine
from config import pword
import datetime as dt

In [3]:
csv_file = "Resources/btc.csv"
bitcoin_data = pd.read_csv(csv_file)
bitcoin_data.head()

,date,txVolume(USD),adjustedTxVolume(USD),txCount,marketcap(USD),price(USD),exchangeVolume(USD),realizedCap(USD),generatedCoins,fees,activeAddresses,averageDifficulty,paymentCount,medianTxValue(USD),medianFee,blockSize,blockCount
0,2009-01-03,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-04,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-06,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-07,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# create dataframe with only columns we want to keep
bitcoin_clean = bitcoin_data[['date', 'price(USD)', 'exchangeVolume(USD)', 'generatedCoins']]

bitcoin_clean.head()

,date,price(USD),exchangeVolume(USD),generatedCoins
0,2009-01-03,NaN,NaN,NaN
1,2009-01-04,NaN,NaN,NaN
2,2009-01-05,NaN,NaN,NaN
3,2009-01-06,NaN,NaN,NaN
4,2009-01-07,NaN,NaN,NaN


In [7]:
# drop rows with null values for price
bitcoin_clean = bitcoin_clean.dropna(subset=['price(USD)'])

bitcoin_clean.head()

,date,price(USD),exchangeVolume(USD),generatedCoins
1576,2013-04-28,135.30,0.0,4425.0000
1577,2013-04-29,134.44,0.0,4349.9988
1578,2013-04-30,144.00,0.0,3725.0000
1579,2013-05-01,139.00,0.0,3775.0000
1580,2013-05-02,116.38,0.0,3350.0000


In [8]:
bitcoin_clean.dtypes

date                    object
price(USD)             float64
exchangeVolume(USD)    float64
generatedCoins         float64
dtype: object

In [11]:
dates = pd.to_datetime(bitcoin_clean['date'])
bitcoin_clean['dt'] = dates
bitcoin_clean = bitcoin_clean[['dt', 'price(USD)', 'exchangeVolume(USD)', 'generatedCoins']]

<ipython-input-11-02d5f95b45c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_clean['dt'] = dates


,dt,price(USD),exchangeVolume(USD),generatedCoins
1576,2013-04-28,135.30,0.0,4425.0000
1577,2013-04-29,134.44,0.0,4349.9988
1578,2013-04-30,144.00,0.0,3725.0000
1579,2013-05-01,139.00,0.0,3775.0000
1580,2013-05-02,116.38,0.0,3350.0000


In [12]:
bitcoin_clean = bitcoin_clean.rename(columns={'dt':'date', 'price(USD)': 'price', 
                                              'exchangeVolume(USD)': 'exchange_volume',
                                             'generatedCoins': 'generated'})
bitcoin_clean.head()

,date,price,exchange_volume,generated
1576,2013-04-28,135.30,0.0,4425.0000
1577,2013-04-29,134.44,0.0,4349.9988
1578,2013-04-30,144.00,0.0,3725.0000
1579,2013-05-01,139.00,0.0,3775.0000
1580,2013-05-02,116.38,0.0,3350.0000


In [13]:
rds_connection_string = f"postgres:{pword}@localhost:5432/ETL_project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [15]:
bitcoin_clean.to_sql(name='bitcoin', con=engine, if_exists='append', index=False)